## Visualize cross

- distribution of detection

In [2]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess

import countpassenger
import pandas as pd
import os.path as osp
import os

df_cross = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "mbk-tourist-raw-cross-object-20240429.csv")
)

/home/jeans/internship/parking-customer-count


In [3]:
df_cross

,timestamp_precise,_id,id,count_id,ymax,ymin,travel_distance,travel_trajectory,original_video_name,xmin,...,timestamp,frame_no,lifetime,true_video_name,camera,travel_displacement,video_name,xmax,full_image_names,tracking_id
0,2024-04-29 16:57:43.839000,662fda258d1d64541a821630,310g0_mbk-14-12_1714409701_1714409863_1310,310g0,1222,452,1624.366708,217.874984,mbk-14-12_1714409701.mp4,330,...,2024-04-29 16:55:01,1310,1.599,mbk-14-12_1714409701.mp4,mbk-14-12,1588.906087,mbk-14-12_1714409701.mp4,789,[],310
1,2024-04-29 16:45:29.997000,662fdd218d1d64541a979cd2,184g0_mbk-14-14_1714409101_1714409129_239,184g0,100,0,436.509887,355.481929,mbk-14-14_1714409101.mp4,2383,...,2024-04-29 16:45:01,239,3.759,mbk-14-14_1714409101.mp4,mbk-14-14,431.119146,mbk-14-14_1714409101.mp4,2449,[],184
2,2024-04-29 16:45:37.555000,662fdbf58d1d64541a8f1499,543g0_mbk-14-13_1714409101_1714409137_299,543g0,480,44,8052.897421,19.271324,mbk-14-13_1714409101.mp4,1754,...,2024-04-29 16:45:01,299,221.342,mbk-14-13_1714409101.mp4,mbk-14-13,507.395290,mbk-14-13_1714409101.mp4,1913,[],543
3,2024-04-29 16:42:21.606000,662fdd128d1d64541a9752f9,155g0_mbk-14-14_1714408801_1714408941_1132,155g0,187,0,768.732021,338.629378,mbk-14-14_1714408801.mp4,2283,...,2024-04-29 16:40:01,1132,2.520,mbk-14-14_1714408801.mp4,mbk-14-14,173.538468,mbk-14-14_1714408801.mp4,2386,[],155
4,2024-04-29 16:44:08.296000,662fdd128d1d64541a975300,176g0_mbk-14-14_1714408801_1714409048_1986,176g0,96,0,588.819043,359.249443,mbk-14-14_1714408801.mp4,2402,...,2024-04-29 16:40:01,1986,6.998,mbk-14-14_1714408801.mp4,mbk-14-14,536.392051,mbk-14-14_1714408801.mp4,2463,[],176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2287,2024-04-29 02:02:52.077000,662f02098d1d64541a90daac,35g0_mbk-14-13_1714356001_1714356172_1376,35g0,1304,601,2297.252498,237.154942,mbk-14-13_1714356001.mp4,377,...,2024-04-29 02:00:01,1376,5.759,mbk-14-13_1714356001.mp4,mbk-14-13,1431.522567,mbk-14-13_1714356001.mp4,941,[],35
2288,2024-04-29 02:02:50.198000,662f02098d1d64541a90dab1,37g0_mbk-14-13_1714356001_1714356170_1361,37g0,979,285,1390.414566,231.166845,mbk-14-13_1714356001.mp4,1108,...,2024-04-29 02:00:01,1361,2.359,mbk-14-13_1714356001.mp4,mbk-14-13,1148.548382,mbk-14-13_1714356001.mp4,1419,[],37
2289,2024-04-29 02:03:45.182000,662f020a8d1d64541a90e0c9,52g0_mbk-14-11_1714356001_1714356225_1800,52g0,615,173,1468.095005,9.462322,mbk-14-11_1714356001.mp4,1740,...,2024-04-29 02:00:01,1800,33.590,mbk-14-11_1714356001.mp4,mbk-14-11,391.780795,mbk-14-11_1714356001.mp4,1923,[],52
2290,2024-04-29 02:03:49.301000,662f020a8d1d64541a90e0d7,54g0_mbk-14-11_1714356001_1714356229_1833,54g0,625,182,1460.334544,335.239691,mbk-14-11_1714356001.mp4,1733,...,2024-04-29 02:00:01,1833,21.754,mbk-14-11_1714356001.mp4,mbk-14-11,631.913130,mbk-14-11_1714356001.mp4,1879,[],54
